## Training demo for any model with custom loss function and SeismicDataLoader

In this demo, SeismicConvLSTM is used as the model and CustomLoss is used as the loss function.
- This demo uses train and validation dataloaders from dataloader.py. If you haven't seen dataloader_demo.ipynb, check it first.
- First, import train and validation dataloader
- Next, import custom loss function from customloss.py
- Next, we add SeismicConvLSTM model for demo purposes
- Next, import torch optimizer (we will use Adam, if you use other optimizers, import it)
- Then, we add matplotlib plt for traning, validation graph plotting
- Finally, add train_model function from trainer.py

In [4]:
from dataloader import train_dataloader, val_dataloader
from customLoss import CustomLoss
from convlstm import SeismicConvLSTM, ModifiedSeismicConvLSTM
import torch.optim as optim
import matplotlib.pyplot as plt
from trainer import train_model, train_model_with_logging

[I] Checking if the dataset is downloaded with the name afad_hfd5.zip
[W] Dataset is not found, downloading the dataset


Downloading...
From (original): https://drive.google.com/uc?id=1isvI3lELKocPuF2mIjMiMpXrcaoWTaIQ
From (redirected): https://drive.google.com/uc?id=1isvI3lELKocPuF2mIjMiMpXrcaoWTaIQ&confirm=t&uuid=e8ca48e9-c6a7-4fcc-8ee2-176d7bb9bc19
To: /home/hekimoglu/workspace/git/gaia-ps-detection/afad_hdf5.zip
100%|██████████| 497M/497M [00:07<00:00, 65.5MB/s] 
/home/hekimoglu/workspace/git/gaia-ps-detection/customDataset.py:53: RuntimeWarning: overflow encountered in scalar multiply
  s_idx = int(s_ts*sampling_rate)


[S] Dataset is downloaded, loading the dataset


/home/hekimoglu/workspace/git/gaia-ps-detection/customDataset.py:47: RuntimeWarning: overflow encountered in scalar multiply
  p_idx = int(p_ts*sampling_rate)


---------------- Dataset information: ----------------
[I] Sample rate: 100 Hz
[I] Chosen window size:900
[I] Hopping size:300
[I] Each sample is window_size/sample rate seconds long. Each sample has 3 channels.
[I] Dataset values shape is: [num_samples, num_window, num_channel, (height) 1, (width) num_sample_points]
[I] Dataset value shape: torch.Size([1575, 3, 3, 1, 900])
[I] Dataset labels shape is: [num_samples, 4], where the 4 values are [p_idx, s_idx, no_PS, only_P, only_S, both_PS]
[I] Dataset label shape:torch.Size([4725, 6])
[I] One sample shape: torch.Size([3, 3, 1, 900])
[I] One label shape: torch.Size([6])
[I] ----------------------------------------------------
[I] Some label examples in the dataset:
[I] Label: tensor([-1., -1.,  1.,  0.,  0.,  0.])
[I] Label: tensor([894.,  -1.,   0.,   1.,   0.,   0.])
[I] Label: tensor([359., 953.,   0.,   0.,   0.,   1.])
[I] Label: tensor([    1286.,       -1.,        0.,        1.,        0.,        0.])
[S] -------------------------

### Model setup

This below cell creates a model instance. If you'd like to use your own model, change this cell.
The example SeismicConvLSTM model takes the below inputs:
- input channel size
- output dimension
- numer of sequence windows
- data points in each window
- hidden dimensions of convlstm cells
- kernel size of conv layers
- number of convlstm cells

model variable is used to keep the model

In [5]:
# Example instantiation of SeismicConvLSTM
input_dim = 3  # Number of input channels
hidden_dims = [32, 32, 64, 64, 128]  # Hidden dimensions for each ConvLSTM cell
kernel_sizes = [(11, 1), (5, 1), (3, 1), (3, 1), (3, 1)]  # Kernel sizes for each cell
strides = [(11, 1), (3, 1), (1, 1), (1, 1), (3, 1)]  # Strides for each ConvLSTM cell
paddings = [(5, 0), (2, 0), (1, 0), (1, 0), (1, 0)]  # Paddings for each ConvLSTM cell
num_layers = 5  # Number of layers
output_dim = 6  # Output dimension [P_index, S_index, P_confidence, S_confidence]
num_windows = 3 #900 # Number of windows (for flattening later)
data_points = 900  #1 # Height is 1
 # self.fc1 = nn.Linear(hidden_dims[-1] * num_windows * 1 * data_points, 512)  # Adjusting dimensions according to hidden_dims[-1]
model = ModifiedSeismicConvLSTM(input_dim, hidden_dims, kernel_sizes, strides, paddings, num_layers, output_dim, num_windows, data_points)

In [6]:
print(model)

ModifiedSeismicConvLSTM(
  (convlstm): ModifiedConvLSTM(
    (cell_list): ModuleList(
      (0): ModifiedConvLSTMCell(
        (conv): Conv2d(35, 128, kernel_size=(11, 1), stride=(11, 1), padding=(5, 0))
      )
      (1): ModifiedConvLSTMCell(
        (conv): Conv2d(64, 128, kernel_size=(5, 1), stride=(3, 1), padding=(2, 0))
      )
      (2): ModifiedConvLSTMCell(
        (conv): Conv2d(96, 256, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
      )
      (3): ModifiedConvLSTMCell(
        (conv): Conv2d(128, 256, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
      )
      (4): ModifiedConvLSTMCell(
        (conv): Conv2d(192, 512, kernel_size=(3, 1), stride=(3, 1), padding=(1, 0))
      )
    )
  )
  (fc2): Linear(in_features=512, out_features=6, bias=True)
)


In [7]:
# # Define the model
# # We are dynamically fetching the input channels, num_windows, data_points_of_each_window, and the output_dim from the dataset using the dataloader
# # Create an iterator from the dataloader
# train_iter = iter(train_dataloader)
# # Get the first batch
# one_sample = next(train_iter) # one_sample[0].shape, one_sample[1].shape --> torch.Size([16, 3, 3, 1, 900]) torch.Size([16, 6])

# INPUT_CHANNELS = one_sample[0].shape[2] # one_sample[0].shape[2] --> 3
# OUTPUT_DIM = one_sample[1].shape[1] # one_sample[1].shape[1] --> 6
# NUM_WINDOWS = one_sample[0].shape[1] # one_sample[0].shape[1] --> 3
# DATA_POINTS_OF_EACH_WINDOW = one_sample[0].shape[4] # one_sample[0].shape[4] --> 900
# HIDDEN_DIM = [32, 32, 64, 64, 128]
# KERNEL_SIZE = (11, 5, 3, 3, 3)   
# NUM_CONVLSTM_BLOCKS = 5
# print("output_dim: ", OUTPUT_DIM)
# assert len(HIDDEN_DIM) == NUM_CONVLSTM_BLOCKS, "The number of hidden dimensions should be equal to the number of ConvLSTM blocks"
# model = SeismicConvLSTM(input_dim=INPUT_CHANNELS, hidden_dim=HIDDEN_DIM, kernel_size=KERNEL_SIZE, num_layers=NUM_CONVLSTM_BLOCKS, output_dim=OUTPUT_DIM, num_windows = NUM_WINDOWS, data_points = DATA_POINTS_OF_EACH_WINDOW)

# print(model)

### Training the model
We have been created dataloaders and model. Now we will train the model with generic train function.
- First, initialize optimizer. In this demo, Adam optimizer is used with lr = LEARNING_RATE.
- Next, set the loss function as criterion. In this demo, CustomLoss function with lambda_val = CUSTOM_LOSS_LAMBDA is used.
- Then, set the number of epochs.
- Next, train the model. train_model function trains the model with train_dataloader, and validates after each epoch with val_dataloader

In [8]:
LEARNING_RATE = 0.2
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

CUSTOM_LOSS_LAMBDA = 2
criterion = CustomLoss(lambda_val=CUSTOM_LOSS_LAMBDA)

num_epochs = 90

# Train the model with the training dataloader
model, train_losses, val_losses = train_model_with_logging(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs, 'train_log.csv', early_stopping=True)

Initialized FC1 with input size: 345600
Epoch 0/89, Training Loss: 376877.0069
Validation Loss: 97627.9271
Epoch 1/89, Training Loss: 104716.6967
Validation Loss: 91508.6604
Epoch 2/89, Training Loss: 101345.4860
Validation Loss: 90014.4966
Epoch 3/89, Training Loss: 101660.4119
Validation Loss: 90204.4732
Epoch 4/89, Training Loss: 101757.0233
Validation Loss: 91719.7502
Early stopping activated
Epoch 5/89, Training Loss: 101538.3714
Validation Loss: 90650.3306
Epoch 6/89, Training Loss: 101564.3403
Validation Loss: 89427.2360
Epoch 7/89, Training Loss: 101425.6986
Validation Loss: 90758.7577
Epoch 8/89, Training Loss: 101397.8369
Validation Loss: 90425.9416
Epoch 9/89, Training Loss: 102314.9195
Validation Loss: 88879.9453
Epoch 10/89, Training Loss: 101469.6757
Validation Loss: 91455.7387
Epoch 11/89, Training Loss: 101882.2809
Validation Loss: 90215.3931


KeyboardInterrupt: 

In [6]:
import os
import time
import torch

# INPUT_CHANNELS = one_sample[0].shape[2] # one_sample[0].shape[2] --> 3
# OUTPUT_DIM = one_sample[1].shape[1] # one_sample[1].shape[1] --> 6
NUM_WINDOWS = 3 # one_sample[0].shape[1] --> 3
DATA_POINTS_OF_EACH_WINDOW = 900 # one_sample[0].shape[4] --> 900
# HIDDEN_DIM = [32, 32, 64, 64, 128]
# KERNEL_SIZE = (11, 5, 3, 3, 3)   
# NUM_CONVLSTM_BLOCKS = 5

os.makedirs('models', exist_ok=True)
date_time_str = time.strftime("%Y%m%d-%H%M")
model_name = 'models/model_{}_epoch{}_lr{}_lambda{}_windows{}_datapoints{}.pth'.format(date_time_str, num_epochs, LEARNING_RATE, CUSTOM_LOSS_LAMBDA, NUM_WINDOWS, DATA_POINTS_OF_EACH_WINDOW)
torch.save(model.state_dict(), model_name)
print('Model saved at: ', model_name)

# plot and save the training and validation losses
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('models/Train_and_val_loss_model_{}'.format(model_name.split('/')[1].split('.')[0]))
plt.show()

Model saved at:  models/model_20241017-1959_epoch90_lr0.2_lambda2_windows3_datapoints900.pth


NameError: name 'train_losses' is not defined